In [1]:
"""
https://github.com/davendw49/k2
https://arxiv.org/abs/2306.05064
We also construct GeoBench, an evaluation dataset comprising more than 1500 objective questions and 939 subjective questions collected from:
National Postgraduate Entrance Examination (NPEE) on Geoscience and AP Test Geology, Geography, and Environmental Science.
"""

!git clone https://github.com/davendw49/k2.git

Cloning into 'k2'...
remote: Enumerating objects: 195, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 195 (delta 37), reused 15 (delta 10), pack-reused 135 (from 1)
Receiving objects: 100% (195/195), 30.02 MiB | 9.60 MiB/s, done.
Resolving deltas: 100% (96/96), done.
Updating files: 100% (35/35), done.


In [2]:
import pandas as pd

aps = pd.read_json("k2/data/geobench/geobench_apstudy.json")
# npee = pd.read_json("k2/data/geobench/geobench_npee.json")

print(aps.keys())
# print(npee.keys())

Index(['id', 'question', 'answerKey'], dtype='object')


In [3]:
def append_text_to_stem_aps(dataframe):

  strings = []
  for index, row in dataframe.iterrows():
    question = row["question"]["stem"]
    answer = [choice["text"] for choice in row["question"]["choices"] if choice["label"] == row["answerKey"]][0]

    strings.append(f"Question: {question}. Answer: {answer}")

  return strings

In [4]:
aps_strings = append_text_to_stem_aps(aps)
print(len(aps_strings))
print(aps_strings[0])

1395
Question: The umbrella theory explaining the Earth's movement, contact, and flattening of large land plates is known as. Answer: plate tectonics


In [5]:
# Write the sentences to the file, each separated by a newline
with open("geobench_ap_perplexity_sample.txt", 'w') as f:
    for sentence in aps_strings:
        f.write(sentence + '\n')

In [6]:
df = pd.DataFrame({'sentence': aps_strings})
df['word_count'] = df['sentence'].apply(lambda x: len(x.split()))

df["word_count"].mean()

21.62078853046595

In [7]:
!pip install tqdm -q
!pip install -U bitsandbytes -q
!pip install transformers accelerate bitsandbytes -q
!pip install datasets -q
!pip install --upgrade huggingface_hub -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [8]:
import torch
import os
from google.colab import userdata
# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import math

from datasets import load_dataset
import nltk

from tqdm import tqdm

In [ ]:
# Check so there is a gpu available, a T4(free tier) is enough to run this notebook
assert (torch.cuda.is_available()==True)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B")
quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3.1-8B",
                                            quantization_config=quantization_config,
                                            device_map='auto')

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [11]:
# Check so there is a gpu available, a T4(free tier) is enough to run this notebook
assert (torch.cuda.is_available()==True)

model_repo = "eve-esa/eve-base-v0.2"
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B")
quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(
    model_repo,
    device_map="auto",   # Uses available GPUs if any
    torch_dtype="auto",  # Automatically sets the correct tensor type
    use_safetensors=True, # Ensures it loads safetensors files
    quantization_config=quantization_config,
)

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/892 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

In [12]:
def calculate_perplexity(model, tokenizer, text):
    """Calculates the perplexity of a given text using a language model.

    Args:
        model: The language model.
        tokenizer: The tokenizer for the language model.
        text: The text to calculate perplexity for.

    Returns:
        The perplexity of the text.

    Example: calculate_perplexity(model, tokenizer, "The quick brown fox jumps over the lazy dog")
    """
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
        perplexity = math.exp(loss.item())
    return perplexity


In [13]:
from google.colab import files
import pandas as pd
import io

uploaded = files.upload()
with open("wiki_perplexity_sample.txt", "r") as f:
  wiki_strings = f.readlines()

Saving wiki_perplexity_sample.txt to wiki_perplexity_sample.txt


In [14]:
# get micro perplexity

micro_aps_ppl = [calculate_perplexity(model, tokenizer, sent) for sent in tqdm(aps_strings)]
ppl_sents = list(zip(micro_aps_ppl, aps_strings))

  0%|          | 0/1395 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
100%|██████████| 1395/1395 [08:05<00:00,  2.87it/s]

62.40399166734406


In [20]:
print(sum(micro_aps_ppl) / len(aps_strings))

62.40399166734406


In [15]:
# save analysis file
df = pd.DataFrame(ppl_sents, columns=["perplexity", "text"])
df.to_csv("eve-base-v0.2_perplexity_text_ap_geobench.csv")

In [17]:
# get micro perplexity on general domain
micro_wiki_ppl = [calculate_perplexity(model, tokenizer, sent) for sent in tqdm(wiki_strings)]
wiki_ppl_sents = list(zip(micro_wiki_ppl, wiki_strings))
print(sum(micro_wiki_ppl) / len(wiki_strings))

100%|██████████| 6000/6000 [38:01<00:00,  2.63it/s]

35.045766961213005


In [18]:
print(sum(micro_wiki_ppl) / len(wiki_strings))

35.045766961213005


In [19]:
# save analysis file on general domain
df = pd.DataFrame(wiki_ppl_sents, columns=["perplexity", "text"])
df.to_csv("eve-base-v0.2_perplexity_text_wiki.csv")

In [22]:
# Meta-Llama-3.1-8B (vanilla)
# 8bit
# micro-ppl ap test QA - 39.861749458454426
# micro-ppl wiki - 29.052969335750973

In [21]:
# Meta-Llama-3.1-8B (CPT'd)
# 8bit
# micro-ppl ap test QA - 62.40399166734406
# micro-ppl wiki - 35.045766961213005